In [53]:
import numpy as np
import pandas as pd
from os.path import join as oj
import os
import pandas as pd
import sys
import inspect
from datetime import datetime, timedelta
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
sys.path.append(parentdir + '/modeling')
import load_data
from fit_and_predict import add_preds
from functions import merge_data
from viz import  viz_interactive, viz_static, viz_map_utils
import plotly.figure_factory as ff
import plotly
import re
import plotly.express as px
from urllib.request import urlopen
import json


#Load data and add prediction
def add_pre(df, var, name, newname):
    h = df_county[var + '1-day'].copy()
    for i in range(2,8):
        h = np.vstack((h,df_county[var + str(i) +'-day']))
    df_county[name] = [h.T[i] for i in range(len(h.T))]



## Add prediction history to dataframe
def add_prediction_history(df_tab):
    def find_interval(a):
        return [max(a[6][0] - a[5][1], 0), max(a[6][1] - a[5][0], 0)]
    def add_predictions_7day(data,df):
        data = data.sort_values(by='countyFIPS')
        dic = {'cases':'Cases','deaths':'Deaths'}
        for i in range(df_tab.shape[0]):
            for key in dic.keys():
                df.loc[i,'pred_7day_'+key].append(data.loc[i,'Predicted '+ dic[key] +' 7-day'])
                df.loc[i,'pred_7day_'+key+'_interval'].append(data.loc[i,'Predicted '+dic[key]+' Intervals'][6])
                df.loc[i,'pred_7day_new_'+key].append(data.loc[i,'Predicted '+ dic[key] +' 7-day']-data.loc[i,'Predicted '+ dic[key] +' 6-day'])
                df.loc[i,'pred_7day_new_'+key+'_interval'].append(find_interval(data.loc[i,'Predicted '+ dic[key] +' Intervals']))

    cached_dir=oj(parentdir, 'data')
    i = 0
    for c in ['deaths','cases']:
        for pre in ['pred_7day_','pred_7day_new_']:
            df_tab[pre + c] =[[] for _ in range(df_tab.shape[0])]
            df_tab[pre + c + '_interval'] =[[] for _ in range(df_tab.shape[0])]
    date2 = []
    k = 0
    while True:
        d = (datetime.today() - timedelta(days=i)).date()
        i += 1
        if cached_dir is not None:
            cached_fname = oj(cached_dir, f'preds_{d.month}_{d.day}_cached.pkl')
            if os.path.exists(cached_fname):
                date2.append(d + timedelta(days = 6))
                add_predictions_7day(pd.read_pickle(cached_fname),df_tab)
            else:
                k += 1
                if k > 1:
                    break
    return df_tab, date2
# generate html for individual counties
def generate_all_counties():
    print('generating html for counties')
    df_tab = df_county[['CountyName', 'State', 'tot_deaths',
             'deaths', 'cases','countyFIPS','pred_cases','pred_deaths','Predicted Deaths Intervals','Predicted Cases Intervals',
             'pred_new_cases','pred_new_deaths','pred_new_cases_interval','pred_new_deaths_interval']]
    df_tab = df_tab.rename(columns={'CountyName': 'County', 'State': 'State',
                                'Predicted Deaths Intervals': 'pred_deaths_interval',
                                'Predicted Cases Intervals': 'pred_cases_interval'})
    dates = viz_map_utils.date_in_data(df_county)
    df_tab, date2 = add_prediction_history(df_tab)
    viz_interactive.viz_curves_all_counties(df_tab, oj(parentdir, 'results/All_counties/'), dates,date2)
    print('succesfully generated all county html')

# Generate map plot 
def generate_map(key):
    print('generating search map')
    with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
        counties = json.load(response)
    fig = px.choropleth(df_county, geojson=counties, locations='countyFIPS', color=np.log(df_county[key]+1),
                            color_continuous_scale =['#F7E8E4','#F5C8BB','#B96D67','#A83C3B',
                                '#8B2222','#5B0D0D','#5A2318'],
                           scope="usa",
                           labels={'tot_cases':'cumulative cases',
                                  'tot_deaths':'cumulative deaths',
                                  'new_cases_last': 'new case',
                                  'new_deaths_last':'new deaths',
                                  'StateName':'State','CountyName':'County',
                                  'tot_deaths_rate':'deaths per 100k',
                                  'tot_cases_rate':'cases per 100k'},
                            hover_data = ['State','CountyName','tot_cases','new_cases','tot_deaths','tot_deaths_rate','tot_cases_rate'],
                   title = 'County level Covid-19 map on ' + (datetime.today()-timedelta(days=1)).strftime('%m-%d'))
    fig.update_layout(coloraxis_colorbar=dict(len=0.75,
                                  title='cases', 
                                  tickvals = [2.302585092994046,4.605170185988092,6.907755278982137,9.210340371976184,11.512925464970229],
                                  ticktext = ['10', '100', '1000', '10000', '10000','100000'],
                                  x=1, y= 0.5))
    ## update the hover information
    for c in ["countyFIPS=%{location}<br>","<br>color=%{z}"]:
        fig['data'][0]['hovertemplate'] =fig['data'][0]['hovertemplate'].replace(c,"")
    fig['data'][0]['hovertemplate'] =fig['data'][0]['hovertemplate'].replace("=",": ")
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.update_layout(
        paper_bgcolor='rgb(0,0,0)',
        plot_bgcolor='rgb(0,0,0)',
        template='plotly_dark',
    )
    fig['layout'].update(width=1000, height=500, autosize=True,title_x=0.3)
    fig.write_image(oj(parentdir,"results/search_map.png"),width=900, height=450)
    print('succesfully generated search map')   


    return plotly.offline.plot(fig,
                include_plotlyjs='https://cdn.plot.ly/plotly-1.54.1.min.js',
                   output_type='div')

# Add new cases/deaths to dataframe():

# Fill the state full name according to their abbreviations#
def fillstate(df):
    us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
    }
    for i in range(df.shape[0]):
        if df.loc[i, "State"] not in us_state_abbrev.values():
            df.loc[i, "State"] = us_state_abbrev[df.loc[i,"StateName"]]

## update the html file (conbine search1 + search2 + search3)
def update_html(map_html):
    id =re.search('<div id="([^ ]*)"',map_html)
    with open(oj(parentdir,'results/template.html'), 'r') as content_file:
        template = content_file.read()
    f = open(oj(parentdir,'results/search.html'),"w+")
    f.write(template.replace("map id to replace",id.group(1)).replace("Part to replace with map",map_html))
    print('succesfully updated search html')
## Add cases/deaths rate to dataframe
def add_rates(df_county):
    for key in ['tot_deaths','tot_cases']:
        df_county[key+'_rate'] = round(df_county[key] / df_county['PopulationEstimate2018']*100000,2)
## Add new cases/deaths to dataframe
def add_new(df_county):
    def get_col_name(key,days):
        return '#'+key + '_' +(datetime.today()-timedelta(days=days)).strftime('%m-%d-%Y')
    for key in ['Cases','Deaths']:
        df_county['new_'+key.lower()+'_last'] =  df_county[get_col_name(key,1)] - df_county[get_col_name(key,2)]
    for key in ['deaths','cases']:
        df_county['new_' + key] = [[] for _ in range(df_county.shape[0])]
        for i in range(df_county.shape[0]):
            df_county.loc[i,'new_'+key].append(df_county.loc[i, key][0])
            for j in range(1, len(df_county.loc[i, key])):
                df_county.loc[i, 'new_'+key]. append(df_county.loc[i, key][j] - df_county.loc[i, key][j-1])
def add_new_pre(df_county, var, name, newname):    
    
    h = df_county[var + '1-day'] - df_county[name]
    for i in range(2,8):
        h = np.vstack((h,df_county[var + str(i) + '-day'] - df_county[var + str(i - 1) + '-day']))
    df_county[newname] = [h.T[i] for i in range(len(h.T))]

    def find_intervals(b, a):
        tmp = [[a[0][0] - b, a[0][1] - b]]
        for i in range(2, len(a)):
            tmp.append([max(a[i][0] - a[i - 1][1], 0), max(a[i][1] - a[i - 1][0], 0)])
        return tmp
    df_county = df_county.sort_values(by=['countyFIPS'])
    df_county[newname+'_interval'] = [find_intervals(df_county.loc[i,name],df_county.loc[i,var+'Intervals']) for i in range(df_county.shape[0])]
    return df_county  
if __name__ == '__main__':
    print('loading data...')
    NUM_DAYS_LIST = [1, 2, 3, 4, 5, 6, 7]
    df_county = load_data.load_county_level(data_dir=oj(parentdir, 'data')).fillna(0)
    df_county = add_preds(df_county, NUM_DAYS_LIST=NUM_DAYS_LIST, cached_dir=oj(parentdir, 'data')) # adds keys like "Predicted Deaths 1-day"

    ## orgnize predicts as array
    
    add_pre(df_county,'Predicted Cases ','pred_cases','pred_new_cases')
    add_pre(df_county,'Predicted Deaths ','pred_deaths','pred_new_deaths')

    ## add new cases/death to dataframe
    add_new(df_county)
    ## Add new cases/deaths predictions and their intervals 
    df_county = add_new_pre(df_county, 'Predicted Cases ', 'tot_cases', 'pred_new_cases')
    df_county = add_new_pre(df_county, 'Predicted Deaths ', 'tot_deaths', 'pred_new_deaths')

    ##fill missing values of some state full names
    fillstate(df_county)
    ## Add cases/deaths rate to the dataframe
    add_rates(df_county)
    #generate_all_counties()
    #map_html = generate_map('tot_cases')
    #update_html(map_html)


loading data...
loading county-level data...
loaded and merged COVID-19 cases/deaths data successfully


In [54]:
from bokeh.sampledata import us_states, us_counties
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh import palettes
from bokeh.models import ColorBar,HoverTool,LinearColorMapper,ColumnDataSource,FixedTicker, LogColorMapper
output_notebook()
import re
import numpy as np
from modeling import fit_and_predict
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import plot
from plotly.subplots import make_subplots
import json
import plotly.express as px
import plotly
import pandas as pd
from datetime import datetime, timedelta

df_tab = df_county[['CountyName', 'State', 'new_cases','new_deaths',
             'deaths', 'cases','countyFIPS','pred_cases','pred_deaths','Predicted Deaths Intervals','Predicted Cases Intervals',
             'pred_new_cases','pred_new_deaths','pred_new_cases_interval','pred_new_deaths_interval']]
df_tab = df_tab.rename(columns={'CountyName': 'County', 'State': 'State',
                                'Predicted Deaths Intervals': 'pred_deaths_interval',
                                'Predicted Cases Intervals': 'pred_cases_interval'})
dates = viz_map_utils.date_in_data(df_county)
df_tab, date2 = add_prediction_history(df_tab)
df_tab['new_deaths']

Loading BokehJS ...

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
3129    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3130    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3131    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3132    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3133    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: new_deaths, Length: 3142, dtype: object

In [55]:
df_tab.loc[0,'deaths']

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  7,  7,  8,  8,  9,  9,  9,
        9, 11, 11, 11, 12])

In [100]:
row = df_tab.iloc[0]
state,county,FIPS = row['State'],row['County']+" County",row['countyFIPS']
date1 = dates
traces = []
keys_curves=['deaths','cases']
red = '179,74,71'
blue = '83,137,190'
credstr ='rgb(' + red + ')'
cbluestr = 'rgb(' + blue + ')'
fill_strings = ['rgba(' + red + ',0.4)','rgba(' + blue + ',0.4)']
color_strings=[credstr,cbluestr]

def make_plot(pre1, pre2, keys, title):
    fig = make_subplots(rows=1, cols=2, specs=[[{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=("Time series with 7 day prediction",
                                    "Time series with historical prediction(7 day)"))
    def cal_coverage(row,date1,date2,key):
        def cal(key, key_inv):
            dic_real = {}
            dic_pre = {}
            for i in range(len(date1)):
                dic_real[date1[i]] = row[key][i]
            for i in range(len(date2)):
                dic_pre[date2[i]] = row[key_inv][i]
            hit = miss = 0
            for d in dic_pre.keys():
                if d in dic_real:
                    if dic_real[d] >= round(dic_pre[d][0],0) and dic_real[d] <= round(dic_pre[d][1],0):
                        hit += 1
                    else:
                        miss += 1
            return (hit/(hit+miss))
        return [cal(key, pre1 + key +'_interval') for key in keys]
    def make_traces(fig, row, pre, date1, date2, r, c, width,show_lengend,keys_curves):
        for j, key_curve in enumerate(keys_curves):
            curve = row[key_curve]
            traces = []
            traces.append(go.Scatter(x=date1,
                    y=curve,
                    showlegend=show_lengend,
                    visible=True, #key == key0,# False, #key == key0,
                    name=key_curve,
                    line=dict(color=color_strings[j], width=width)))
            low = np.hstack((curve[-1],[x[0] for x in row[pre+key_curve+'_interval']]))
            curve_pre = np.hstack((curve[-1],row[pre+key_curve]))
            up = np.hstack((curve[-1],[x[1] for x in row[pre+key_curve+'_interval']]))
            if pre == pre1:
                low = low[1:]
                up = up[1:]
                curve = curve[1:]
            traces.append(go.Scatter(
                    name = 'Upper Bound',
                    showlegend= False,
                    x = date2,
                    y = up,
                    marker = dict(size=0),
                    line=dict(dash ='solid',color=fill_strings[j], width=0)))
            traces.append(go.Scatter(x= date2,
                    y=curve_pre,
                    name = key_curve +' prediction',
                    showlegend= show_lengend,
                    mode = 'lines',
                    line=dict(dash ='dash',color=color_strings[j], width=width),
                    fillcolor = fill_strings[j],
                    fill ='tonexty'))
            traces.append(go.Scatter(
                    name = 'Lower Bound',
                    showlegend= False,
                    marker = dict(size=0),
                    x = date2,
                    y = low,
                    fill ='tonexty',
                    fillcolor = fill_strings[j],
                    line=dict(dash ='solid',color=fill_strings[j], width=0)))
            for trace in traces:
                fig.add_trace(trace, secondary_y="deaths" in key_curve, row=r,col=c)
        return fig
    newdates = [date1[-1] + timedelta(days =i) for i in range(0,8)]
    fig = make_traces(fig, row,pre2,date1,newdates,1,1,4,True, keys)  
    fig = make_traces(fig, row,pre1,date1,date2,1,2,3,False, keys)

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=15,color='white')  

    deaths_cov,cases_cov = cal_coverage(row,date1,date2,keys)
    # Update the margins to add a title and see graph x-labels.
    fig.layout.update(margin = {'t':120,'b':120})
    if title:        
        fig.layout.update(
            title = dict(text = county+', '+state+ '<br>' + 'FIPS:'+row['countyFIPS'], 
                    y = 0.92,
                    x = 0.03, 
                    yanchor = 'top'),
            legend=dict(x=0.25, y=1.2),
            legend_orientation="h")
    else:
        fig.layout.update(
            legend=dict(x=0.15, y=1.2),
            legend_orientation="h")
    fig.layout.update(
            width=1000,height=600,
            font=dict(size=12)
                )
    fig.add_annotation(dict(font=dict(color="white",size=11),
                            x=0.85,
                            y=0.9,
                            showarrow=False,
                            text='Cases Prediction coverage: '+str(round(cases_cov, 2))+'<br>'
                                '  Deaths Prediction coverage: '+str(round(deaths_cov, 2)),
                            textangle=0,
                            xref="paper",
                            yref="paper"
                           ))
    fig.add_annotation(dict(font=dict(color="white",size=12),
                            x=0.9,
                            y=1.3,
                            showarrow=False,
                            text='Modebar '+u'\u2191',
                            textangle=0,
                            xref="paper",
                            yref="paper"
                           ))
    ## Set plot1 axes properties
    y1 = max([x[1] for x in row[pre2+keys[1]+'_interval']])
    y1 = max(y1, row[keys[1]])
    y2 = max([x[1] for x in row[pre2+keys[0]+'_interval']])
    dic = {'cases':'Cases','deaths':'Deaths','new_cases':'New cases','new_deaths':'New deaths'}
    fig.update_xaxes(title_text="Date",range=[datetime(2020, 3, 24), newdates[-1]],domain = [0,0.41],row=1, col=1)
    fig.update_yaxes(title_text=dic[keys[1]], color=cbluestr,rangemode = 'tozero',
            dtick = y1/5,range=[0,y1], domain=[0,0.95],row=1, col=1)  
    fig.update_yaxes(title_text=dic[keys[0]],showgrid=False,rangemode = 'tozero',color=credstr,
            dtick = y2*1.3/5, range=[0,y2*1.3], secondary_y=True, row=1,col=1)
    ## Set plot2 axes properties
    fig.update_xaxes(title_text="Date",range=[date2[-1], date2[7]],domain = [0.57,0.98], row=1, col=2)
    fig.update_yaxes(title_text=dic[keys[1]], color=cbluestr,rangemode = 'tozero',
            dtick = y1/5,range=[0,y1], domain=[0,0.95],row=1, col=2)  
    fig.update_yaxes(title_text=dic[keys[0]],rangemode = 'tozero',color=credstr,
            dtick = y2*1.3/5, showgrid=False,range=[0,y2*1.3], secondary_y=True, row=1,col=2)
    fig.layout.template = 'plotly_dark'
    return plotly.offline.plot(fig, include_plotlyjs= "https://cdn.plot.ly/plotly-1.54.1.min.js", output_type='div')

    plotly.offline.plot(fig, filename='test_direct.html', auto_open=True)

In [101]:
s1 = make_plot('pred_7day_','pred_',['deaths','cases'], True)
s2 = make_plot('pred_7day_','pred_',['new_deaths','new_cases'], False)

In [102]:
f = open('test.html',"w+")
f.write(s1+s2)

69521

In [ ]:
content = plotly.offline.plot(fig, include_plotlyjs= "https://cdn.plot.ly/plotly-1.54.1.min.js", output_type='div')
with open(oj(parentdir,'results/template.html'), 'r') as content_file:
        template = content_file.read()
f = open(oj(parentdir,'results/test.html'),"w+")
f.write(content)

In [ ]:
def generate_map(key):
    print('generating search map')
    fips = df_county['countyFIPS']
    values = df_county[key]
    binning = []
    level = 6
    for i in range(level):
        binning.append(np.rint(2**(np.log2(max(values))/(level+1)*(i+1))))
    fig = ff.create_choropleth(fips=fips, values=values,
                           colorscale = ['#F7E8E4','#F5C8BB','#B96D67','#A83C3B',
                                '#8B2222','#5B0D0D','#5A2318'],binning_endpoints=binning,
                           legend_title='Cumulative Cases on '+(datetime.today()-timedelta(days=1)).strftime('%m-%d')
                          )

    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,255)',
        plot_bgcolor='rgba(0,0,0,255)',
        template='plotly_dark'
    )
    fig['layout'].update(width=1000, height=500, autosize=True)

    fig.write_image(oj(parentdir,"results/search_map'+'_key'.png"),width=900, height=450)
    plotly.offline.plot(fig, include_plotlyjs='https://cdn.plot.ly/plotly-1.42.3.min.js',filename=oj(parentdir,"results/search_map.html"), auto_open=True)
    print('succesfully generated search map')
    plotly.offline.plot(fig,
                include_plotlyjs='https://cdn.plot.ly/plotly-1.42.3.min.js',
                   output_type='div')
    return fig

In [ ]:
figure = generate_map('tot_cases')


In [ ]:
# Modify tooltip
hover_trace = [t for t in fig['data'] if 'text' in t]
for i, label in enumerate(hover_trace['text']):
    
    # Remove FIPS
    new_label = label.replace("FIPS: %s<br>" % fips[i], "")
    
    # Add a new value
    #new_label += "<br>Other: %d" % values2[i]
    
    # Update trace text
    hover_trace['text'][i] = new_label


In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


import plotly.express as px
def generate_map(key):
    fig = px.choropleth(df_county, geojson=counties, locations='countyFIPS', color=np.log(df_county[key]+1),
                            color_continuous_scale =['#F7E8E4','#F5C8BB','#B96D67','#A83C3B',
                                '#8B2222','#5B0D0D','#5A2318'],
                           scope="usa",
                           labels={'tot_cases':'cumulative cases',
                                  'tot_deaths':'cumulative deaths',
                                  'new_cases': 'new case',
                                  'new_deaths':'new deaths',
                                  'StateName':'State','CountyName':'County',
                                  'tot_deaths_rate':'deaths per 100k',
                                  'tot_cases_rate':'cases per 100k'},
                            hover_data = ['State','CountyName','tot_cases','new_cases','tot_deaths','tot_deaths_rate','tot_cases_rate'],
                   title = 'Cumulative Cases on ' + (datetime.today()-timedelta(days=1)).strftime('%m-%d'))
    fig.update_layout(coloraxis_colorbar=dict(len=0.75,
                                  title='cases', 
                                  tickvals = [2.302585092994046,4.605170185988092,6.907755278982137,9.210340371976184,11.512925464970229],
                                  ticktext = ['10', '100', '1000', '10000', '10000','100000'],
                                  x=1, y= 0.5))
## update the hover information
    for c in ["countyFIPS=%{location}<br>","<br>color=%{z}"]:
        fig['data'][0]['hovertemplate'] =fig['data'][0]['hovertemplate'].replace(c,"")
    fig['data'][0]['hovertemplate'] =fig['data'][0]['hovertemplate'].replace("=",": ")
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.update_layout(
        paper_bgcolor='rgb(0,0,0)',
        plot_bgcolor='rgb(0,0,0)',
        template='plotly_dark',
    )
    fig['layout'].update(width=1000, height=500, autosize=True)
    fig.update_layout(title_x=0.3)

    return plotly.offline.plot(fig,
                include_plotlyjs='https://cdn.plot.ly/plotly-1.54.1.min.js',
                   output_type='div')

In [ ]:
#dic  = {'tot_cases':'Cumulative Cases','tot_deaths':"Cumulative Deaths",'new_cases':"New Cases",'new_deaths':'New Deaths'}
#for k in dic.keys():
 #   generate_map(k, dic[k])
    
search2 = generate_map('tot_cases')

In [116]:
c = df_county[df_county['State'] == 'Alaska']['CountyName'].values
print(c)

['Aleutians East' 'Aleutians West' 'Anchorage' 'Bethel' 'Bristol Bay'
 'Denali' 'Dillingham' 'Fairbanks North Star' 'Haines'
 'Hoonah–Angoon Census Area' 'Juneau' 'Kenai Peninsula'
 'Ketchikan Gateway' 'Kodiak Island' 'Lake and Peninsula'
 'Matanuska-Susitna' 'Nome' 'North Slope' 'Northwest Arctic'
 'Petersburg Borough' 'Prince of Wales-Hyder Census Area' 'Sitka'
 'Skagway Municipality' 'Southeast Fairbanks' 'Valdez-Cordova'
 'Wade Hampton' 'Wrangell City and Borough' 'Yakutat' 'Yukon-Koyukuk']


In [118]:
for i in range(len(c)):
    print(c[i] + ' Borough')

Aleutians East Borough
Aleutians West Borough
Anchorage Borough
Bethel Borough
Bristol Bay Borough
Denali Borough
Dillingham Borough
Fairbanks North Star Borough
Haines Borough
Hoonah–Angoon Census Area Borough
Juneau Borough
Kenai Peninsula Borough
Ketchikan Gateway Borough
Kodiak Island Borough
Lake and Peninsula Borough
Matanuska-Susitna Borough
Nome Borough
North Slope Borough
Northwest Arctic Borough
Petersburg Borough Borough
Prince of Wales-Hyder Census Area Borough
Sitka Borough
Skagway Municipality Borough
Southeast Fairbanks Borough
Valdez-Cordova Borough
Wade Hampton Borough
Wrangell City and Borough Borough
Yakutat Borough
Yukon-Koyukuk Borough


In [133]:
for i in range(len(c)):
    print('form.options['+str(i+1)+'] = new Option("'+c[i]+ ' Borough''","'+c[i]+ ' Borough''");')

form.options[1] = new Option("Aleutians East Borough","Aleutians East Borough");
form.options[2] = new Option("Aleutians West Borough","Aleutians West Borough");
form.options[3] = new Option("Anchorage Borough","Anchorage Borough");
form.options[4] = new Option("Bethel Borough","Bethel Borough");
form.options[5] = new Option("Bristol Bay Borough","Bristol Bay Borough");
form.options[6] = new Option("Denali Borough","Denali Borough");
form.options[7] = new Option("Dillingham Borough","Dillingham Borough");
form.options[8] = new Option("Fairbanks North Star Borough","Fairbanks North Star Borough");
form.options[9] = new Option("Haines Borough","Haines Borough");
form.options[10] = new Option("Hoonah–Angoon Census Area Borough","Hoonah–Angoon Census Area Borough");
form.options[11] = new Option("Juneau Borough","Juneau Borough");
form.options[12] = new Option("Kenai Peninsula Borough","Kenai Peninsula Borough");
form.options[13] = new Option("Ketchikan Gateway Borough","Ketchikan Gateway 

In [132]:
max(1,max(df_county.loc[1,'deaths']))

10